# Для тренировки модели нужно установить turicreate
Linux or mac
python 3.6 - 3.8

`pip install turicreate`

### На вход:
- _attend_features.csv (с нашими фичами)
- _groups.csv

### Выход:
- predicts/predict_v01.csv


In [1]:
import turicreate as tc
import pandas as pd

In [2]:
actions = tc.SFrame.read_csv('_attend_features.csv')

Finished parsing file /media/vasiliev/SSD_Data/_dev/ezee_may_2003/ds/_attend_features.csv

Parsing completed. Parsed 100 lines in 0.327077 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,float,str,str,str,float,str,str,str,str,float,float,str,int,str,float,float,str,int,float,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 144983 lines. Lines per second: 423167

Finished parsing file /media/vasiliev/SSD_Data/_dev/ezee_may_2003/ds/_attend_features.csv

Parsing completed. Parsed 600390 lines in 1.09762 secs.

### Делим общую таблицу на фичи для attend, users, groups

In [3]:
print(actions.head)
actions1 = actions[['group_id',
                    'user_id',
                    'log_visits',
                    #'user_lat',
                    #'user_long',
                    'направление 1',
                    'направление 2',
                    'направление 3',
                    'weekend']]

users_1 = actions[['user_id',
                   'sex',
                   'age',
                   'user_address',
                   'user_lat',
                   'user_long',
                  'registration_first_attend_delta',
                  'is_newbee']]

groups_1 = actions[['group_id',
                    'lat',
                    'long',
                    'направление 1',
                    'направление 2',
                    'направление 3',
                    'is_online',
                    'weekend',
                    'lesson_length',
                    'session']]

<bound method SFrame.head of Columns:
	X1	int
	group_id	int
	user_id	int
	log_visits	float
	направление 3	str
	is_online	str
	weekend	str
	lesson_length	float
	session	str
	направление 1	str
	направление 2	str
	group_address	str
	lat	float
	long	float
	sex	str
	age	int
	user_address	str
	user_lat	float
	user_long	float
	first_date	str
	registration_first_attend_delta	int
	deviation_from_delta	float
	is_newbee	str

Rows: 600390

Data:
+----+-----------+-----------+--------------------+---------------+-----------+
| X1 |  group_id |  user_id  |     log_visits     | направление 3 | is_online |
+----+-----------+-----------+--------------------+---------------+-----------+
| 0  | 801346549 | 101370217 | 2.0794415416798357 |  Здорово жить |  offline  |
| 1  | 801346549 | 101375025 | 3.044522437723423  |  Здорово жить |  offline  |
| 2  | 801346549 | 101392971 |        0.0         |  Здорово жить |  offline  |
| 3  | 801346550 | 101352023 | 4.1588830833596715 |  ОНЛАЙН Цигун |   online  |
| 

There are a variety of machine learning techniques that can be used to build a recommender model. Turi Create provides a method turicreate.recommender.create that will automatically choose an appropriate model for your data set. More details on choosing a model: https://apple.github.io/turicreate/docs/userguide/recommender/choosing-a-model.html

First we create a random split of the data to produce a validation set that can be used to evaluate the model.

### Чему учим модель

Мы учим модель предсказывать логарифм от количества посещений юзером курса (логарифм потому что разброс посещений курса пользователем высокий - от 1 до 64)

### Создание и обучение дефолтной модели

In [4]:
training_data, validation_data = tc.recommender.util.random_split_by_user(actions1, 'user_id', 'group_id', item_test_proportion = 0.2)

model = tc.recommender.create(training_data,
                              'user_id',
                              'group_id',
                              target = 'log_visits',
                              user_data = users_1,
                              item_data = groups_1
                             )


Preparing data set.

Data has 597961 observations with 50906 users and 24535 items.

Data prepared in: 0.695784s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.23885

Final training RMSE: 0.426428

In [5]:
# параметры модели
model

Class                            : RankingFactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : group_id
Target                           : log_visits
Additional observation features  : 4
User side features               : ['user_id', 'sex', 'age', 'user_address', 'user_lat', 'user_long', 'registration_first_attend_delta', 'is_newbee']
Item side features               : ['group_id', 'lat', 'long', 'направление 1', 'направление 2', 'направление 3', 'is_online', 'weekend', 'lesson_length', 'session']

Statistics
----------
Number of observations           : 597961
Number of users                  : 50906
Number of items                  : 24535

Training summary
----------------
Training time                    : 18.7226

Model Parameters
----------------
Model class                      : RankingFactorizationRecommender
num_factors                      : 32
binary_target                    : 0
side_data_factorization       

### Получение предсказаний от дефолтной модели

In [6]:
default_recommendations = model.recommend()

recommendations finished on 1000/50906 queries. users per second: 754.299

recommendations finished on 2000/50906 queries. users per second: 760.594

recommendations finished on 3000/50906 queries. users per second: 765.757

recommendations finished on 4000/50906 queries. users per second: 769.272

recommendations finished on 5000/50906 queries. users per second: 770.237

...

recommendations finished on 46000/50906 queries. users per second: 763.698

recommendations finished on 47000/50906 queries. users per second: 741.329

recommendations finished on 48000/50906 queries. users per second: 719.912

recommendations finished on 49000/50906 queries. users per second: 701.615

recommendations finished on 50000/50906 queries. users per second: 685.712

### Посмотрим рекоммендации для отдельных юзеров

In [7]:
df_groups = pd.read_csv('groups.csv')

def display_history_and_predictions_for_user(user_id, train_dataset, predictions):
    print('----------------------------------------------------------------------------')
    print('User', user_id, 'history:')
    display(train_dataset[train_dataset['user_id'] == user_id][
        'group_id','is_online','session','направление 1','направление 2'])
    predictions = predictions[predictions['user_id'] == user_id].to_dataframe()
    print('User recommendations:')
    predicted_groups = predictions['group_id'].to_list()
    display(df_groups[df_groups['уникальный номер'].isin(predicted_groups)].drop(columns=[
        'адрес площадки','расписание в активных периодах','расписание в закрытых периодах','расписание в плановом периоде']))

In [8]:
user_id = 101352882

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101352882 history:


group_id,is_online,session,направление 1,направление 2
801348234,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801348838,offline,mornig,Физическая активность,Гимнастика
801348944,offline,mornig,Танцы,Танцы
801349048,offline,mornig,Физическая активность,ОФП
801350227,online,mornig,Творчество,ОНЛАЙН Художественно-прикладное творчество ...
801350380,online,mornig,Танцы,ОНЛАЙН Танцы
801351413,online,mornig,Рисование,ОНЛАЙН Рисование
801358464,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801358465,online,mornig,Танцы,ОНЛАЙН Танцы
801358466,online,mornig,Физическая активность,ОНЛАЙН Гимнастика


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
479,801351925,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,Северо-Восточный административный округ,муниципальный округ Марфино
1038,801349295,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,"Южный административный округ, Южный администра...","муниципальный округ Чертаново Южное, муниципал..."
2346,801353512,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN
2415,801350382,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN
3534,801348920,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN
3535,801348420,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,Троицкий и Новомосковский административные округа,городской округ Троицк
3971,801350767,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN
5473,801348444,Творчество,ОНЛАЙН Художественно-прикладное творчество,ОНЛАЙН Рукоделие и творчество,Восточный административный округ,муниципальный округ Северное Измайлово
5534,801349280,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,Центральный административный округ,муниципальный округ Якиманка
10872,801349259,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,Южный административный округ,муниципальный округ Орехово-Борисово Северное


In [12]:
user_id = 101354253

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101354253 history:


group_id,is_online,session,направление 1,направление 2
801347802,online,mornig,Физическая активность,ОНЛАЙН ОФП
801348667,online,evening,Физическая активность,ОНЛАЙН Гимнастика
801348794,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801349796,offline,mornig,Физическая активность,Гимнастика
801349875,offline,aftennoon,Физическая активность,Гимнастика
801350556,online,mornig,Физическая активность,ОНЛАЙН ОФП
801350568,online,mornig,Физическая активность,ОНЛАЙН ОФП
801361065,offline,mornig,Рисование,Рисование


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
707,801353134,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН ОФП,"Центральный административный округ, Центральны...","муниципальный округ Басманный, муниципальный о..."
870,801347110,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН ОФП,"Западный административный округ, Западный адми...","муниципальный округ Можайский, муниципальный о..."
1058,801353163,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН ОФП,"Центральный административный округ, Центральны...","муниципальный округ Басманный, муниципальный о..."
1708,801347075,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН Физкультурно-оздоровительные занятия,Северо-Западный административный округ,муниципальный округ Щукино
1715,801349571,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН Физкультурно-оздоровительные занятия,Северо-Западный административный округ,муниципальный округ Строгино
1760,801351258,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Оздоровительная гимнастика,"Западный административный округ,","муниципальный округ Ново-Переделкино,"
1762,801351112,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН Физкультурно-оздоровительные занятия,Западный административный округ,муниципальный округ Ново-Переделкино
1769,801348457,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН Физкультурно-оздоровительные занятия,"Северный административный округ,","муниципальный округ Сокол,"
2252,801348352,Игры,ОНЛАЙН Интеллектуальные игры,ОНЛАЙН Иные интеллектуальные игры,NaN,NaN
5617,801352452,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН Лечебная физкультура,Западный административный округ,муниципальный округ Фили-Давыдково


In [9]:
user_id = 101385865

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101385865 history:


group_id,is_online,session,направление 1,направление 2
801346923,online,mornig,Танцы,ОНЛАЙН Танцы
801348309,online,mornig,Творчество,ОНЛАЙН Красота и стиль
801348946,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801349093,online,mornig,Образование,ОНЛАЙН Иностранные языки
801350488,offline,aftennoon,Физическая активность,"Фитнес, тренажеры"
801350739,offline,mornig,Танцы,Танцы
801350759,offline,mornig,Танцы,Танцы
801351106,online,mornig,Творчество,ОНЛАЙН Красота и стиль
801351519,offline,mornig,Танцы,Танцы
801351888,offline,mornig,Танцы,Танцы


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
405,801346953,Творчество,ОНЛАЙН Домоводство,ОНЛАЙН Садоводство,Восточный административный округ,муниципальный округ Северное Измайлово
598,801355664,Творчество,ОНЛАЙН Литература,ОНЛАЙН Литература,"Северный административный округ, Северный адми...","муниципальный округ Коптево, муниципальный окр..."
966,801348571,Творчество,ОНЛАЙН Литература,ОНЛАЙН Литература,Восточный административный округ,муниципальный округ Вешняки
1967,801347570,Спецпроект / Московский театрал,ОНЛАЙН Московский театрал,ОНЛАЙН Московский театрал,Северный административный округ,муниципальный округ Дмитровский
1993,801350138,Спецпроект / Московский театрал,ОНЛАЙН Московский театрал,ОНЛАЙН Московский театрал,Северный административный округ,муниципальный округ Дмитровский
1995,801347251,Спецпроект / Московский театрал,ОНЛАЙН Московский театрал,ОНЛАЙН Московский театрал,Северный административный округ,муниципальный округ Дмитровский
2005,801346597,Спецпроект / Московский театрал,ОНЛАЙН Московский театрал,ОНЛАЙН Московский театрал,Северный административный округ,муниципальный округ Дмитровский
2522,801352271,Творчество,ОНЛАЙН Красота и стиль,ОНЛАЙН Школа макияжа,Южный административный округ,муниципальный округ Москворечье-Сабурово
2978,801349625,Творчество,ОНЛАЙН Домоводство,ОНЛАЙН Садоводство,NaN,NaN
3114,801352433,Образование,ОНЛАЙН Здорово жить,ОНЛАЙН Правильное питание,NaN,NaN


### Делаем сравнение различных моделей

Описание всех моделей, заложенных в turicreate, доступно по ссылке:

https://apple.github.io/turicreate/docs/api/turicreate.toolkits.recommender.html

In [10]:
print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain default model')
model_0 = tc.recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                user_data = users_1, item_data = groups_1)

print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain item_similarity model 1')
model_1 = tc.item_similarity_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                                user_data = users_1, item_data = groups_1)

print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain item_similarity model 2')
model_1 = tc.item_similarity_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                                user_data = users_1, item_data = groups_1)


print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain popularity_recommender model')
model_2 = tc.popularity_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                           user_data = users_1, item_data = groups_1)

print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain factorization_recommender model')
model_3 = tc.factorization_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                              user_data = users_1, item_data = groups_1)

print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain ranking_factorization_recommender model')
model_4 = tc.ranking_factorization_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                                      user_data = users_1, item_data = groups_1)

⬛⬛⬛⬛⬛⬛⬛---------------
Train default model


Preparing data set.

Data has 597961 observations with 50906 users and 24535 items.

Data prepared in: 0.673031s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.371674

Final training RMSE: 0.55959

In [11]:
tc.recommender.util.compare_models(
    validation_data, 
    [model_0, model_1,model_2,model_3,model_4], 
    model_names=["⬛⬛⬛⬛⬛⬛⬛ Default model",
                 "⬛⬛⬛⬛⬛⬛⬛ Item similarity model",
                 '⬛⬛⬛⬛⬛⬛⬛ Popularity recommender',
                 '⬛⬛⬛⬛⬛⬛⬛ Factorization recommender',
                 '⬛⬛⬛⬛⬛⬛⬛ Ranking factorization recommender'])

PROGRESS: Evaluate model ⬛⬛⬛⬛⬛⬛⬛ Default model

Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.001388888888888889 | 0.0006944444444444445 |
|   2    |  0.001388888888888889 | 0.0008487654320987653 |
|   3    |  0.001851851851851851 | 0.0010416666666666662 |
|   4    | 0.0013888888888888885 | 0.0010416666666666669 |
|   5    | 0.0013888888888888892 |  0.002430555555555556 |
|   6    | 0.0011574074074074073 |  0.002430555555555557 |
|   7    |  0.001388888888888889 | 0.0034722222222222233 |
|   8    |  0.001215277777777778 |  0.003472222222222222 |
|   9    | 0.0012345679012345677 |  0.003935185185185186 |
|   10   | 0.0011111111111111105 |  0.003935185185185184 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0803796479089258

Per User RMSE (bes

[{'precision_recall_by_user': Columns:
  	user_id	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 12960
  
  Data:
  +-----------+--------+-----------+--------+-------+
  |  user_id  | cutoff | precision | recall | count |
  +-----------+--------+-----------+--------+-------+
  | 101415127 |   1    |    0.0    |  0.0   |   6   |
  | 101415127 |   2    |    0.0    |  0.0   |   6   |
  | 101415127 |   3    |    0.0    |  0.0   |   6   |
  | 101415127 |   4    |    0.0    |  0.0   |   6   |
  | 101415127 |   5    |    0.0    |  0.0   |   6   |
  | 101415127 |   6    |    0.0    |  0.0   |   6   |
  | 101415127 |   7    |    0.0    |  0.0   |   6   |
  | 101415127 |   8    |    0.0    |  0.0   |   6   |
  | 101415127 |   9    |    0.0    |  0.0   |   6   |
  | 101415127 |   10   |    0.0    |  0.0   |   6   |
  +-----------+--------+-----------+--------+-------+
  [12960 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(

`Default model:             `Overall RMSE: 1.06 -> 1.02

`Item similarity model:     `Overall RMSE: 1.93 -> 1.98

`Popularity recommender:    `Overall RMSE: 0.98 -> 0.99

`Factorization recommender: `Overall RMSE: 1.14 -> 1.08

`Ranking factorization:     `Overall RMSE: 1.01 -> 1.07

### Делаем предсказание для Test (с выборкой предсказаний на март-апрель 2023)

In [13]:
import json
with open('groups2023.json') as json_file:
    groups2023 = json.load(json_file)

df_test = pd.read_csv('test.csv')
df_test.head()
df_test['уникальный номер участника'] = df_test['уникальный номер участника'].astype(int)


def display_history_and_predictions_for_user2(user_id, train_dataset, predictions):
    print('----------------------------------------------------------------------------')
    print('User', user_id, 'history:')
    #display(train_dataset[train_dataset['user_id'] == user_id]['group_id','is_online','session','направление 1','направление 2'].to_dataframe())
    display(train_dataset[train_dataset['user_id'] == user_id].to_dataframe())
    predictions = predictions[predictions['user_id'] == user_id]
    print('User recommendations:')
    predicted_groups = predictions['group_id'].to_list()
    display(df_groups[df_groups['уникальный номер'].isin(predicted_groups)].drop(columns=[
        'адрес площадки',
        'расписание в активных периодах',
        'расписание в закрытых периодах',
        'расписание в плановом периоде']))

In [14]:
results = [] 


def get_groups_mix(user_id):
    # Predictions
    x0 = model_0.recommend(users=[user_id], k=1000).to_dataframe()
    x0 = x0[x0['group_id'].isin(groups2023)]
    x0 = x0.head(10)
    
    x1 = model_1.recommend(users=[user_id], k=1000).to_dataframe()
    x1 = x1[x1['group_id'].isin(groups2023)]
    x1 = x1.head(5)
    
    x2 = model_2.recommend(users=[user_id], k=1000).to_dataframe()
    x2 = x2[x2['group_id'].isin(groups2023)]
    x2 = x2.head(10)
    
    x3 = model_3.recommend(users=[user_id], k=1000).to_dataframe()
    x3 = x3[x3['group_id'].isin(groups2023)]
    x3 = x3.head(1)
    
    x4 = model_4.recommend(users=[user_id], k=1000).to_dataframe()
    x4 = x4[x4['group_id'].isin(groups2023)]
    x4 = x4.head(2)
    
    # Отбираем 10 штук, чтобы 1 была от 3 алгоритма и 2 от 4 алгоритма
    x = pd.concat([x1, x4, x0])
    x = x.drop_duplicates(subset='group_id').head(10) 
    
    return x
    
       

for index, row in df_test.iterrows():
    # Получаем юзера
    user_id = row['уникальный номер участника']
    
    x = get_groups_mix(user_id)
    
    display_history_and_predictions_for_user2(user_id, training_data, x)

    groups_list = x['group_id'].to_list()
    groups_list = [str(x) for x in groups_list]    
    groups_str = ','.join(groups_list)
    results.append(groups_str)
 

----------------------------------------------------------------------------
User 801349115 history:


,group_id,user_id,log_visits,направление 1,направление 2,направление 3,weekend


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
1138,801349838,Образование,"ОНЛАЙН История, искусство, краеведение",ОНЛАЙН История искусства,"Восточный административный округ, Восточный ад...","муниципальный округ Северное Измайлово, муници..."
5069,801351551,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северо-Восточный административный округ,муниципальный округ Лианозово
6147,801348860,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северный административный округ,муниципальный округ Ховрино
6549,801351222,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северный административный округ,муниципальный округ Коптево
13902,801360666,Образование,Здорово жить,Здоровый образ жизни (ЗОЖ),Юго-Западный административный округ,муниципальный округ Академический
20256,801368112,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Религиоведение,Троицкий и Новомосковский административные округа,городской округ Троицк
21472,801371717,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21483,801363694,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21484,801369799,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21491,801371145,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN


----------------------------------------------------------------------------
User 801349115 history:


...

,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
1138,801349838,Образование,"ОНЛАЙН История, искусство, краеведение",ОНЛАЙН История искусства,"Восточный административный округ, Восточный ад...","муниципальный округ Северное Измайлово, муници..."
5069,801351551,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северо-Восточный административный округ,муниципальный округ Лианозово
6147,801348860,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северный административный округ,муниципальный округ Ховрино
6549,801351222,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северный административный округ,муниципальный округ Коптево
13902,801360666,Образование,Здорово жить,Здоровый образ жизни (ЗОЖ),Юго-Западный административный округ,муниципальный округ Академический
20256,801368112,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Религиоведение,Троицкий и Новомосковский административные округа,городской округ Троицк
21472,801371717,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21483,801363694,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21484,801369799,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21491,801371145,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN


----------------------------------------------------------------------------
User 801348571 history:


,group_id,user_id,log_visits,направление 1,направление 2,направление 3,weekend


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
1138,801349838,Образование,"ОНЛАЙН История, искусство, краеведение",ОНЛАЙН История искусства,"Восточный административный округ, Восточный ад...","муниципальный округ Северное Измайлово, муници..."
5069,801351551,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северо-Восточный административный округ,муниципальный округ Лианозово
6147,801348860,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северный административный округ,муниципальный округ Ховрино
6549,801351222,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Северный административный округ,муниципальный округ Коптево
13902,801360666,Образование,Здорово жить,Здоровый образ жизни (ЗОЖ),Юго-Западный административный округ,муниципальный округ Академический
20256,801368112,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Религиоведение,Троицкий и Новомосковский административные округа,городской округ Троицк
21472,801371717,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21483,801363694,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21484,801369799,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN
21491,801371145,Образование,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,NaN,NaN


In [15]:
df_test['уникальный номер группы'] = results
df_test.to_csv('test.csv', index=False)

In [16]:
display(df_test)

,,,,,,,,,уникальный номер участника,уникальный номер группы
101346610,801357672,801356697,801356295,801364452,801362733,801356753,801369654,801348754,801349115,"801369799,801363694,801371145,801371717,801349..."
101346612,801367517,801367330,801357672,801370545,801356697,801356753,801354267,801349762,801349115,"801369799,801363694,801371145,801371717,801349..."
101347269,801362915,801366107,801367303,801363184,801366122,801356753,801371688,801349115,801349493,"801369799,801363694,801371145,801371717,801349..."
101347321,801357672,801356697,801369164,801364452,801356295,801356753,801347579,801348754,801349533,"801369799,801363694,801371145,801371717,801349..."
101347983,801358230,801357696,801357632,801361014,801358226,801356753,801369654,801349115,801350123,"801369799,801363694,801371145,801371717,801349..."
...,...,...,...,...,...,...,...,...,...,...
101447423,801373394,801363757,801373402,801363203,801364875,801371688,801371946,801350794,801349115,"801369799,801363694,801371145,801371717,801349..."
101447935,801366606,801362980,801362993,801363085,801364989,801369654,801373288,801349115,801349506,"801369799,801363694,801371145,801371717,801349..."
101449373,801362727,801372933,801365938,801363146,801353958,801369654,801371688,801350123,801349115,"801369799,801363694,801371145,801371717,801349..."
101449505,801369799,801363694,801371145,801371717,801349838,801356753,801371688,801355664,801348571,"801369799,801363694,801371145,801371717,801349..."


### Делаем предсказание для всех пользователей

In [25]:
df_test_all = pd.read_csv('users.csv')
df_test_all = df_test_all[['уникальный номер']]
df_test_all.rename(columns={'уникальный номер':'уникальный номер участника'}, inplace=True)

In [26]:
display(df_test_all)

,уникальный номер участника
0,101391104
1,101437383
2,101351338
3,101354578
4,101411997
...,...
52329,101389206
52330,101355680
52331,101414122
52332,101428464


In [27]:
results = []
for index, row in df_test_all.iterrows():
    # Получаем юзера
    user_id = row['уникальный номер участника']
    
    x = get_groups_mix(user_id)
    
    #display_history_and_predictions_for_user2(user_id, training_data, x)

    groups_list = x['group_id'].to_list()
    groups_list = [str(x) for x in groups_list]    
    groups_str = ','.join(groups_list)
    results.append(groups_str)

In [28]:
df_test_all['уникальный номер группы'] = results
df_test_all.to_csv('test_all.csv', index=False)